##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 量子化認識トレーニングの総合ガイド

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

Keras 量子化認識トレーニングの総合ガイドへようこそ。

このページでは、さまざまなユースケースを示し、それぞれで API を使用する方法を説明します。どの API が必要であるかを特定したら、[API ドキュメント](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/quantization)でパラメータと詳細を確認してください。

- 量子化認識トレーニングのメリットとサポート対象を確認するには、「[概要](https://www.tensorflow.org/model_optimization/guide/quantization/training.md)」をご覧ください。
- 単一のエンドツーエンドの例については、「[量子化認識トレーニングの例](https://www.tensorflow.org/model_optimization/guide/quantization/training_example.md)」をご覧ください。

次のユースケースについて説明しています。

- 以下のステップで、8 ビット量子化でモデルをデプロイします。
    - 量子化認識モデルを定義します。
    - Keras HDF5 モデルの場合のみ、特殊なチェックポイント設定と逆シリアル化ロジックを使用します。そうでない場合、トレーニングは標準どおりです。
    - 量子化認識モデルから量子化モデルを作成します。
- 量子化による実験。
    - すべての実験には、デプロイをサポートするパスはありません。
    - カスタム Keras レイヤーは実験に該当します。

## MNIST モデルをビルドする

必要な API の特定と目的の理解については、次を実行できますが、このセクションを読まずに進むことができます。

In [ ]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import tempfile

input_shape = [20]
x_train = np.random.randn(1, 20).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=20)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(20, input_shape=input_shape),
      tf.keras.layers.Flatten()
  ])
  return model

def setup_pretrained_weights():
  model= setup_model()

  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )

  model.fit(x_train, y_train)

  _, pretrained_weights = tempfile.mkstemp('.tf')

  model.save_weights(pretrained_weights)

  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

setup_model()
pretrained_weights = setup_pretrained_weights()

##量子化認識モデルを定義する

以下のようにしてモデルを定義した場合、バックエンドにデプロイするために使用できるパスが[概要ページ](https://www.tensorflow.org/model_optimization/guide/quantization/training.md)に一覧表示されています。デフォルトでは、8 ビット量子化が使用されます。

注意: 量子化認識モデルは、実際には量子化されていません。量子化モデルの作成は別のステップで行われます。

### モデル全体を量子化する

**ユースケース:**

- サブクラス化モデルはサポートされていません。

**モデルの精度を高めるための<strong>ヒント</strong>:**

- 精度を最も低下させるレイヤーの量子化を省略する場合は、「一部のレイヤーを量子化する」を試してください。
- 一般的に、ゼロからトレーニングするのではなく、量子化認識トレーニングでファインチューニングする方が優れています。


モデル全体を量子化認識にするには、モデルに `tfmot.quantization.keras.quantize_model` を適用します。


In [ ]:
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()

### 一部のレイヤーを量子化する

モデルを量子化すると、精度に悪影響が及ぶことがあります。モデルのレイヤーを選択的に量子化すると、精度、速度、およびモデルサイズ間のトレードオフを調べることができます。

**ユースケース:**

- 完全に量子化されたモデル（EdgeTPU v1、ほとんどの DSP など）でのみ機能するバックエンドをデプロイするには、「モデル全体を量子化する」をご覧ください。

**モデルの精度を高めるための<strong>ヒント</strong>:**

- 一般的に、ゼロからトレーニングするのではなく、量子化認識トレーニングでファインチューニングする方が優れています。
- 最初のレイヤーではなく、後のレイヤーを量子化してみてください。
- クリティカルレイヤー（注意機構など）を量子化しないようにしてください。


以下の例では、`Dense` レイヤーのみを量子化しています。

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

# Helper function uses `quantize_annotate_layer` to annotate that only the 
# Dense layers should be quantized.
def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` 
# to the layers of the model.
annotated_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_quantization_to_dense,
)

# Now that the Dense layers are annotated,
# `quantize_apply` actually makes the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

この例では量子化するものを決定するためにレイヤーの種類が使用されていますが、特定のレイヤーを量子化する上で最も簡単な方法は、`name` プロパティを設定し、`clone_function` でその名前を探す方法です。

In [ ]:
print(base_model.layers[0].name)

#### 可読性を高められても、モデルの精度を潜在的に低下させる

これは、量子化認識トレーニングによるファインチューニングとは使用できないため、上記の例よりも精度に劣る可能性があります。

**Functional の例**

In [ ]:
# Use `quantize_annotate_layer` to annotate that the `Dense` layer
# should be quantized.
i = tf.keras.Input(shape=(20,))
x = tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(10))(i)
o = tf.keras.layers.Flatten()(x)
annotated_model = tf.keras.Model(inputs=i, outputs=o)

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

# For deployment purposes, the tool adds `QuantizeLayer` after `InputLayer` so that the
# quantized model can take in float inputs instead of only uint8.
quant_aware_model.summary()

**Sequential の例**


In [ ]:
# Use `quantize_annotate_layer` to annotate that the `Dense` layer
# should be quantized.
annotated_model = tf.keras.Sequential([
  tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=input_shape)),
  tf.keras.layers.Flatten()
])

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

## チェックポイントと逆シリアル化

**ユースケース:** このコードは、HDF5 モデル形式のみで必要です（HDF5 重みまたはその他の形式では不要です）。

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
quant_aware_model.save(keras_model_file)

# `quantize_scope` is needed for deserializing HDF5 models.
with tfmot.quantization.keras.quantize_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## 量子化モデルを作成してデプロイする

通常は、使用するデプロイバックエンドのドキュメントを参照してください。

これは、TFLite バックエンドの例です。

In [ ]:
base_model = setup_pretrained_model()
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Typically you train the model here.

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

## 量子化による実験

**ユースケース**: 以下の API を使用した場合、サポートされているデプロイパスはありません。たとえば、TFLite 変換とカーネル実装は 8 ビット量子化のみをサポートしています。特徴は実験的であり、下位互換性はありません。

- `tfmot.quantization.keras.QuantizeConfig`
- `tfmot.quantization.keras.quantizers.Quantizer`
- `tfmot.quantization.keras.quantizers.LastValueQuantizer`
- `tfmot.quantization.keras.quantizers.MovingAverageQuantizer`

### セットアップ: DefaultDenseQuantizeConfig

実験には、レイヤーの重み、活性化、および出力を量子化する方法を記述した `tfmot.quantization.keras.QuantizeConfig` を使用する必要があります。

以下は、API デフォルトで `Dense` レイヤーに使用されるのと同じ `QuantizeConfig` を定義する例です。

この例では順伝播中に、`get_weights_and_quantizers` に返される `LastValueQuantizer` が入力として `layer.kernel` に呼び出され、出力を生成します。この出力は元の `Dense` レイヤーの順伝播で `set_quantize_weights` に定義されたロジックに従って `layer.kernel` に置き換えられます。これと同じ考え方が活性化と出力に適用されます。


In [ ]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

### カスタム Keras レイヤーを量子化する


この例では、`DefaultDenseQuantizeConfig` を使用して `CustomLayer` を量子化します。

構成の適用は、「量子化による実験」のユースケースと同じです。

- `tfmot.quantization.keras.quantize_annotate_layer` を `CustomLayer` に適用し、`QuantizeConfig` を渡します。
- `tfmot.quantization.keras.quantize_annotate_model` を使用して、API のデフォルトでモデルの残りを量子化し続けます。


In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class CustomLayer(tf.keras.layers.Dense):
  pass

model = quantize_annotate_model(tf.keras.Sequential([
   quantize_annotate_layer(CustomLayer(20, input_shape=(20,)), DefaultDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'DefaultDenseQuantizeConfig': DefaultDenseQuantizeConfig,
   'CustomLayer': CustomLayer}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### 量子化パラメータを変更する


**一般的な過ち:** 32 ビット未満にバイアスを量子化すると、通常、モデルの精度を著しく悪化させてしまいます。

この例は、`Dense` レイヤーが重みにデフォルトの 8 ビットではなく 4 ビットを使用するように変更します。モデルの残りは、API のデフォルトを使用し続けます。


In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=4, symmetric=True, narrow_range=False, per_axis=False))]

構成の適用は、「量子化による実験」のユースケースと同じです。

- `tfmot.quantization.keras.quantize_annotate_layer` を `Dense` レイヤーに適用し、`QuantizeConfig` を渡します。
- `tfmot.quantization.keras.quantize_annotate_model` を使用して、API のデフォルトでモデルの残りを量子化し続けます。

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this Dense layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### 量子化するレイヤーの一部を変更する


この例は、`Dense` レイヤーが活性化の量子化をスキップするように変更します。モデルの残りは、API のデフォルトを使用し続けます。

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
      # Skip quantizing activations.
      return []

    def set_quantize_activations(self, layer, quantize_activations):
      # Empty since `get_activaations_and_quantizers` returns
      # an empty list.
      return

構成の適用は、「量子化による実験」のユースケースと同じです。

- `tfmot.quantization.keras.quantize_annotate_layer` を `Dense` レイヤーに適用し、`QuantizeConfig` を渡します。
- `tfmot.quantization.keras.quantize_annotate_model` を使用して、API のデフォルトでモデルの残りを量子化し続けます。

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this Dense layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### カスタム量子化アルゴリズムを使用する


`tfmot.quantization.keras.quantizers.Quantizer` クラスは、入力に任意のアルゴリズムを適用できるコーラブルです。

この例では、入力は重みであり、`FixedRangeQuantizer` __call__ 関数の計算を重みに適用します。元の重みの値の代わりに、`FixedRangeQuantizer` の出力が、その重みを使用したものすべてに渡されます。

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class FixedRangeQuantizer(tfmot.quantization.keras.quantizers.Quantizer):
  """Quantizer which forces outputs to be between -1 and 1."""

  def build(self, tensor_shape, name, layer):
    # Not needed. No new TensorFlow variables needed.
    return {}

  def __call__(self, inputs, training, weights, **kwargs):
    return tf.keras.backend.clip(inputs, -1.0, 1.0)

  def get_config(self):
    # Not needed. No __init__ parameters to serialize.
    return {}


class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      # Use custom algorithm defined in `FixedRangeQuantizer` instead of default Quantizer.
      return [(layer.kernel, FixedRangeQuantizer())]

構成の適用は、「量子化による実験」のユースケースと同じです。

- `tfmot.quantization.keras.quantize_annotate_layer` を `Dense` レイヤーに適用し、`QuantizeConfig` を渡します。
- `tfmot.quantization.keras.quantize_annotate_model` を使用して、API のデフォルトでモデルの残りを量子化し続けます。

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()